In [12]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# Load in the cleaned data
metaPath = 'Data/cleanedMeta.csv'
reviewPath = 'Data/cleanedReviews.csv'

metaDF = pd.read_csv(metaPath, index_col = 0, keep_default_na = False)
reviewDF = pd.read_csv(reviewPath, index_col = 0, keep_default_na = False)

In [14]:
metaDF.head()

,category,description,title,also_buy,brand,asin
0,"['Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'Economics & Economic Theory']","['REASON is edited for people interested in economic, social, and international issues. Viewpoint stresses individual liberty, private responsibility, and limited government. Some emphasis on Pacific Rim, local/state issues with national impact, science/technology. Regular departments include news/trends, book reviews (mostly history, politics, and economics), and cultural commentary.']",,"['B002PXVYLE', 'B01MCU84LB', 'B000UHI2LW', 'B01AKS14AQ', 'B002PXVYPU', 'B001THPA26', '1476752842', 'B002CT515Q', 'B00KQ0HP2K', 'B00XZF1JUM', 'B0058EONOM', 'B01FV51RKA', 'B0032KHQTS', 'B002GCU2S0', 'B079JCLNZ4', 'B002PXW18E', 'B00005NIOH', 'B00005N7SD', 'B002LDA9VY', '0345816021', 'B00006KX3K', 'B002GCU2SA', 'B0047VIALE', 'B00007AZRH', 'B06XWB4X41', '0465058728', 'B0045VPFDW', 'B00FP59VGY', 'B001THPA3K', 'B002BFZ9NA', '0735224897', 'B01F2MKW0I', 'B0036QY66Y', '0415922224', '0226315398', 'B00005N7R5', 'B002KUNVD6', 'B00005N7PT', 'B0025ZOVEO', 'B002PXW0M6', 'B002PXVYEQ', 'B002PXVYGO', 'B002PXVYVE', '1548650218', '0465028616', 'B01FIR6AJ4', '1541645375', 'B003I7HS3Y', '0061253804', '1491904224', '1944424768', 'B001TP73UU', 'B00EV5HI0K', '0062308270', '0525540075', 'B06XWC6PRS', 'B002PXW0LW', 'B002PXVYBE', 'B001THPA58', 'B004GVZUUY', 'B001TP73UA', 'B000063XJR', 'B002PXVYO6', '0465060730', '1594204225', 'B002PXW24C', 'B002XPVNOC', 'B002CT51A6', '0998289027', '0061479012', '0525427570', 'B001U5SPLK']",Reason Magazine,B00005N7NQ
1,"['Arts, Music & Photography', 'Music']","['Written by and for musicians. Covers a variety of musical styles and includes transcriptions from recordings and solo pieces for guitar.', '', '']",,"['B002PXVYGE', 'B0054LRNC8', 'B000BVEELE', 'B00006KC3U', 'B00S5KZHCS', 'B000OMG0KI', '1719064873', '1495028798', '1979584729', 'B002PXVYUU', 'B003MXE4JU', 'B000O1PKOG', 'B0032KHQTS', 'B003HNY1IY', 'B01MCU84LB', 'B002LDA9VY', 'B000UHI2LW', 'B001THPA58', 'B002PXVYEQ', 'B002PXVZ6I', '0634024051']",String Letter Publishers,B00005N7OC
2,"['Fashion & Style', 'Women']","['Allure is the beauty expert. Every issue is full of celebrity tips and insider secrets from the pros, like what works overnight and what works for a lifetime. Editors pick their favorite new products and reveal what new styles really work for you. Your subscription includes the annual special issues: Makeovers and Best of Beauty.', '', '']",,"['B001THPA4O', 'B002PXVZWW', 'B001THPA1M', 'B001THPA12', 'B01N819UDU', 'B001TP73UA', 'B001U5SPLK', 'B001THPA4Y', 'B06XWC6PRS', 'B0029XHGRQ', 'B002PXW0EO', 'B0039L2QLS', 'B001THPA3A', 'B00O85C28Q', 'B002ANPG4U', 'B001THPA1C', 'B004GVZUU4', 'B002PXW24C', 'B001TP73UU', 'B07574N7PW', 'B002VB3X96', 'B001THPA3K', 'B079JCLNZ4', 'B0029XHGTY', 'B0029XHGSK', 'B001THPA44', 'B06XW52KRF', 'B002PXW06C', 'B002PXVYVE', 'B001THPA1W', 'B004GVZUUY', 'B002XPVNOC', 'B0061SHL48', 'B002PXVYBE', 'B001THPA3U', 'B0007INI2C', 'B002PXVYO6', 'B01N702LLM', 'B0046Z4TZW', 'B0089FUF6W', 'B06XWB4X41', 'B00M9NU5BU', 'B001THPA26', 'B001TP73TG', 'B002BFZ9MQ', 'B002CT51A6', 'B00KRPPEWW', 'B002V0CJL0', 'B0023BNL1E', 'B0029XHGVM', 'B0061SHL16', 'B0757G8K2G', 'B001THPA58', 'B002CT51EM', 'B0032KHQTS', 'B00005NIOH', 'B002CT515Q', 'B002PXVYEQ', 'B00KQ0HP2K', 'B003N3QRF8', 'B002PXVZPO', 'B002PXVZO0', 'B000O1PKOG', 'B001TP73UK', 'B005W1DVCG', 'B002GCU2SA', 'B07C2TG9DP', 'B0742LVLMJ', 'B0029XHGSU', 'B000063XJL', 'B00007AZWJ', 'B00413TVTS', 'B01MCU84LB', 'B078XGW32W', 'B07BNNZRD2', 'B07J1VT854']",Conde Nast Publications,B00005N7OD
3,"['Sports, Recreation & Outdoors', 'Sports & Leisure', 'Flying']","['FLIGHT JOURNAL includes articles on aviation history and contemporary practice; military, commercial and civil aircraft; notable personalities of aviation; technical aspects of aircraft; visits to air shows and museums; reports on restorations; aviation photography; three-view aircraft drawings; aviation art

In [15]:
metaDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2320 entries, 0 to 3384
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     2320 non-null   object
 1   description  2320 non-null   object
 2   title        2320 non-null   object
 3   also_buy     2320 non-null   object
 4   brand        2320 non-null   object
 5   asin         2320 non-null   object
dtypes: object(6)
memory usage: 126.9+ KB


In [16]:
reviewDF.head()

,overall,reviewerID,asin
0,5.0,AH2IFH762VY5U,B00005N7P0
1,5.0,AOSFI0JEYU4XM,B00005N7P0
2,3.0,A3JPFWKS83R49V,B00005N7OJ
3,5.0,A19FKU6JZQ2ECJ,B00005N7OJ
4,5.0,A25MDGOMZ2GALN,B00005N7P0


In [17]:
reviewDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84167 entries, 0 to 89686
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   overall     84167 non-null  float64
 1   reviewerID  84167 non-null  object 
 2   asin        84167 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.6+ MB


In [8]:
# Look at sample cell values in metaDF 
pd.set_option('display.max_colwidth', None)
metaDF.iloc[[0]]

,category,description,title,also_buy,brand,asin
0,"['Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'Economics & Economic Theory']","['REASON is edited for people interested in economic, social, and international issues. Viewpoint stresses individual liberty, private responsibility, and limited government. Some emphasis on Pacific Rim, local/state issues with national impact, science/technology. Regular departments include news/trends, book reviews (mostly history, politics, and economics), and cultural commentary.']",NaN,"['B002PXVYLE', 'B01MCU84LB', 'B000UHI2LW', 'B01AKS14AQ', 'B002PXVYPU', 'B001THPA26', '1476752842', 'B002CT515Q', 'B00KQ0HP2K', 'B00XZF1JUM', 'B0058EONOM', 'B01FV51RKA', 'B0032KHQTS', 'B002GCU2S0', 'B079JCLNZ4', 'B002PXW18E', 'B00005NIOH', 'B00005N7SD', 'B002LDA9VY', '0345816021', 'B00006KX3K', 'B002GCU2SA', 'B0047VIALE', 'B00007AZRH', 'B06XWB4X41', '0465058728', 'B0045VPFDW', 'B00FP59VGY', 'B001THPA3K', 'B002BFZ9NA', '0735224897', 'B01F2MKW0I', 'B0036QY66Y', '0415922224', '0226315398', 'B00005N7R5', 'B002KUNVD6', 'B00005N7PT', 'B0025ZOVEO', 'B002PXW0M6', 'B002PXVYEQ', 'B002PXVYGO', 'B002PXVYVE', '1548650218', '0465028616', 'B01FIR6AJ4', '1541645375', 'B003I7HS3Y', '0061253804', '1491904224', '1944424768', 'B001TP73UU', 'B00EV5HI0K', '0062308270', '0525540075', 'B06XWC6PRS', 'B002PXW0LW', 'B002PXVYBE', 'B001THPA58', 'B004GVZUUY', 'B001TP73UA', 'B000063XJR', 'B002PXVYO6', '0465060730', '1594204225', 'B002PXW24C', 'B002XPVNOC', 'B002CT51A6', '0998289027', '0061479012', '0525427570', 'B001U5SPLK']",Reason Magazine,B00005N7NQ


From the metadata, I will extract the following features:
- category
- description
- brand

In [18]:
metaDF[['category']].value_counts()

category                                                                                                  
                                                                                                              442
['Lifestyle & Cultures']                                                                                       66
['Professional & Educational Journals', 'Professional & Trade', 'Transportation']                              55
['Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'History']     55
['Travel, City & Regional', 'United States', 'South']                                                          50
                                                                                                             ... 
['Entertainment & Pop Culture', 'Performing Arts']                                                              1
['Science, History & Nature', 'Mathematics']                                                   

In [19]:
metaDF[['description']].value_counts()

description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [20]:
metaDF[['brand']].value_counts()

brand                              
Hearst Magazines                       255
Conde Nast                             184
                                       147
Meredith                                48
Bonnier Corporation                     43
                                      ... 
Archaeological Institute of America      1
Hearst Magazines Italia Spa              1
Heavy Horse World                        1
Heavy Metal                              1
Zyzzyva /Laura Cogan                     1
Length: 943, dtype: int64

In [22]:
# Combining relevant features into a single feature
features = ['category', 'description', 'brand']
def combined_features(row):
    combined_features = ''
    for feature in features:
        combined_features = combined_features + ' ' + row[feature] 
    return combined_features

metaDF['combined_features'] = metaDF.apply(combined_features, axis = 1)
metaDF.head()

,category,description,title,also_buy,brand,asin,combined_features
0,"['Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'Economics & Economic Theory']","['REASON is edited for people interested in economic, social, and international issues. Viewpoint stresses individual liberty, private responsibility, and limited government. Some emphasis on Pacific Rim, local/state issues with national impact, science/technology. Regular departments include news/trends, book reviews (mostly history, politics, and economics), and cultural commentary.']",,"['B002PXVYLE', 'B01MCU84LB', 'B000UHI2LW', 'B01AKS14AQ', 'B002PXVYPU', 'B001THPA26', '1476752842', 'B002CT515Q', 'B00KQ0HP2K', 'B00XZF1JUM', 'B0058EONOM', 'B01FV51RKA', 'B0032KHQTS', 'B002GCU2S0', 'B079JCLNZ4', 'B002PXW18E', 'B00005NIOH', 'B00005N7SD', 'B002LDA9VY', '0345816021', 'B00006KX3K', 'B002GCU2SA', 'B0047VIALE', 'B00007AZRH', 'B06XWB4X41', '0465058728', 'B0045VPFDW', 'B00FP59VGY', 'B001THPA3K', 'B002BFZ9NA', '0735224897', 'B01F2MKW0I', 'B0036QY66Y', '0415922224', '0226315398', 'B00005N7R5', 'B002KUNVD6', 'B00005N7PT', 'B0025ZOVEO', 'B002PXW0M6', 'B002PXVYEQ', 'B002PXVYGO', 'B002PXVYVE', '1548650218', '0465028616', 'B01FIR6AJ4', '1541645375', 'B003I7HS3Y', '0061253804', '1491904224', '1944424768', 'B001TP73UU', 'B00EV5HI0K', '0062308270', '0525540075', 'B06XWC6PRS', 'B002PXW0LW', 'B002PXVYBE', 'B001THPA58', 'B004GVZUUY', 'B001TP73UA', 'B000063XJR', 'B002PXVYO6', '0465060730', '1594204225', 'B002PXW24C', 'B002XPVNOC', 'B002CT51A6', '0998289027', '0061479012', '0525427570', 'B001U5SPLK']",Reason Magazine,B00005N7NQ,"['Professional & Educational Journals', 'Professional & Trade', 'Humanities & Social Sciences', 'Economics & Economic Theory'] ['REASON is edited for people interested in economic, social, and international issues. Viewpoint stresses individual liberty, private responsibility, and limited government. Some emphasis on Pacific Rim, local/state issues with national impact, science/technology. Regular departments include news/trends, book reviews (mostly history, politics, and economics), and cultural commentary.'] Reason Magazine"
1,"['Arts, Music & Photography', 'Music']","['Written by and for musicians. Covers a variety of musical styles and includes transcriptions from recordings and solo pieces for guitar.', '', '']",,"['B002PXVYGE', 'B0054LRNC8', 'B000BVEELE', 'B00006KC3U', 'B00S5KZHCS', 'B000OMG0KI', '1719064873', '1495028798', '1979584729', 'B002PXVYUU', 'B003MXE4JU', 'B000O1PKOG', 'B0032KHQTS', 'B003HNY1IY', 'B01MCU84LB', 'B002LDA9VY', 'B000UHI2LW', 'B001THPA58', 'B002PXVYEQ', 'B002PXVZ6I', '0634024051']",String Letter Publishers,B00005N7OC,"['Arts, Music & Photography', 'Music'] ['Written by and for musicians. Covers a variety of musical styles and includes transcriptions from recordings and solo pieces for guitar.', '', ''] String Letter Publishers"
2,"['Fashion & Style', 'Women']","['Allure is the beauty expert. Every issue is full of celebrity tips and insider secrets from the pros, like what works overnight and what works for a lifetime. Editors pick their favorite new products and reveal what new styles really work for you. Your subscription includes the annual special issues: Makeovers and Best of Beauty.', '', '']",,"['B001THPA4O', 'B002PXVZWW', 'B001THPA1M', 'B001THPA12', 'B01N819UDU', 'B001TP73UA', 'B001U5SPLK', 'B001THPA4Y', 'B06XWC6PRS', 'B0029XHGRQ', 'B002PXW0EO', 'B0039L2QLS', 'B001THPA3A', 'B00O85C28Q', 'B002ANPG4U', 'B001THPA1C', 'B004GVZUU4', 'B002PXW24C', 'B001TP73UU', 'B07574N7PW', 'B002VB3X96', 'B001THPA3K', 'B079JCLNZ4', 'B0029XHGTY', 'B0029XHGSK', 'B001THPA44', 'B06XW52KRF', 'B002PXW06C', 'B002PXVYVE', 'B001THPA1W', 'B004GVZUUY', 'B002XPVNOC', 'B0061SHL48', 'B002PXVYBE', 'B001THPA3U', 'B0007INI2C', 'B002PXVYO6', 'B01N702LLM', 'B0046Z4TZW', 'B0089FUF6W', 'B06XWB4X41', 'B00M9NU5BU', 'B001THPA26', 'B001TP73TG', 'B002BFZ9MQ', 'B002CT51A6', 'B00KRPPEWW', 'B002V0CJL0', 'B0023BNL1E', 'B0029XHGVM', 'B0061SHL16'

In [23]:
# Extracting features 
cv = CountVectorizer()
count_matrix = cv.fit_transform(metaDF['combined_features'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [24]:
#cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix)
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,...,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319
0,1.000000,0.248186,0.242124,0.198999,0.367173,0.271589,0.244068,0.294811,0.267318,0.208972,...,0.271295,0.271295,0.245774,0.248015,0.190330,0.043621,0.102719,0.196150,0.127589,0.170935
1,0.248186,1.000000,0.352357,0.271453,0.335870,0.302968,0.267912,0.235432,0.282147,0.196057,...,0.249889,0.249889,0.224820,0.279478,0.225689,0.000000,0.156603,0.199363,0.111154,0.186145
2,0.242124,0.352357,1.000000,0.257742,0.347524,0.463952,0.388091,0.318896,0.272475,0.152145,...,0.567048,0.567048,0.325669,0.347011,0.200160,0.123858,0.166667,0.396059,0.103510,0.265794
3,0.198999,0.271453,0.257742,1.000000,0.272848,0.244629,0.307776,0.214966,0.361173,0.260623,...,0.325819,0.325819,0.243514,0.388514,0.247630,0.078581,0.277568,0.235571,0.197013,0.197958
4,0.367173,0.335870,0.347524,0.272848,1.000000,0.402651,0.408466,0.321390,0.316540,0.228416,...,0.437256,0.437256,0.339534,0.391578,0.250418,0.114783,0.220099,0.289044,0.143889,0.250603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2315,0.043621,0.000000,0.123858,0.078581,0.114783,0.207079,0.195325,0.087773,0.167134,0.080403,...,0.332174,0.332174,0.322695,0.269680,0.237999,1.000000,0.073398,0.327035,0.205129,0.274817
2316,0.102719,0.156603,0.166667,0.277568,0.220099,0.200628,0.197126,0.088582,0.155700,0.304290,...,0.228246,0.228246,0.139573,0.299382,0.133440,0.073398,1.000000,0.165025,0.230022,0.123267
2317,0.196150,0.199363,0.396059,0.235571,0.289044,0.409718,0.384268,0.142090,0.300623,0.144620,...,0.616548,0.616548,0.456052,0.485071,0.451869,0.327035,0.165025,1.000000,0.286972,0.398195
2318,0.127589,0.111154,0.103510,0.197013,0.143889,0.134985,0.107123,0.176048,0.145048,0.251976,...,0.265789,0.265789,0.423783,0.456383,0.430946,0.205129,0.230022,0.286972,1.000000,0.325363


We can see that the cosine similarity scores on the diagonals are 1, this means that product 0 is 100% similar to itself.  We can see that product 0 is more similar to product 4 (cosine similarity score: 0.3672) than to product 3 (cosine similarity score: 0.1990). 

## Reviewer 'A5RHZE7B8SV5Q'

In [48]:
# Let's look at reviewer 'A5RHZE7B8SV5's preferences
reviewer = reviewDF[reviewDF.reviewerID == 'A5RHZE7B8SV5Q'].sort_values('overall')
reviewer

,overall,reviewerID,asin
426,1.0,A5RHZE7B8SV5Q,B00005N7P0
23329,2.0,A5RHZE7B8SV5Q,B00005NIPX
3752,4.0,A5RHZE7B8SV5Q,B00005N7S5
70,5.0,A5RHZE7B8SV5Q,B00005N7PS
9239,5.0,A5RHZE7B8SV5Q,B00005N7TL
11880,5.0,A5RHZE7B8SV5Q,B00005N7QI
21963,5.0,A5RHZE7B8SV5Q,B00005NIND
24778,5.0,A5RHZE7B8SV5Q,B00005QDWQ


In [106]:
pd.set_option('display.max_colwidth', None)

# Let's look at magazine 'B00005NIND' which was rated 5 stars by the reviewer
metaDF[metaDF['asin'] == 'B00005N7PS']

category  \
65  ['Fashion & Style', 'Men']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [102]:
# Inputting into the model
# Let's see what our recommendation system recommends for someone who likes the above magazine
magazine_user_likes = 'B00005N7PS'

def get_index_from_asin(asin):
    # Get the index of the product from it's asin
    return metaDF[metaDF.asin == asin].index.values[0]

magazine_index = get_index_from_asin(magazine_user_likes)

In [103]:
# Generate similar magazines matrix
similar_magazines = list(enumerate(cosine_sim[magazine_index]))

In [104]:
#sort similar magazines list in descending order
sorted_similar_magazines = sorted(similar_magazines, key=lambda x:x[1], reverse=True)
pd.DataFrame(sorted_similar_magazines)

,0,1
0,65,1.000000
1,126,0.806188
2,193,0.800578
3,1296,0.796873
4,1177,0.790495
...,...,...
2315,2143,0.000000
2316,2203,0.000000
2317,2253,0.000000
2318,2268,0.000000


In [108]:
# Print top 5 most similar magazines

pd.set_option('display.max_colwidth', None)


def get_magazine_from_index(index):
    recs = [metaDF.iloc[index].loc['asin'], metaDF.iloc[index].loc['description'], (metaDF.iloc[index].loc['category'])]
    if metaDF.iloc[index].loc['brand']:
        recs.append(metaDF.iloc[index].loc['brand'])
    else:
        recs.append(metaDF.iloc[index].loc['title'])
    return recs

recsList = []
i = 0
for magazine in sorted_similar_magazines[1:]:
    recsList.append(get_magazine_from_index(magazine[0]))
    i += 1
    if i > 4:
        break

recsDF = pd.DataFrame(recsList, columns = ['asin', 'description', 'category', 'brand'])
recsDF

asin  \
0  B00005NIPX   
1  B00005UQ67   
2  B000LXHJFK   
3  B000EGCIW8   
4  B00005N7QC   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  